In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 98% !important }</style>"))

In [2]:
from utilities import *
from models import *

import joblib

In [3]:
pin_file = "../Data/pin.csv"
pin = read_pin(pin_file)

In [4]:
enc = joblib.load("../Models/MLP_Classification_Rolling_FFill_MinMax_Encoder.joblib")

In [5]:
model = joblib.load('../Models/RF_Classification_Rolling_FFill_MinMax.joblib')

In [6]:
client = InfluxDBClient()

In [7]:
data = client.retrieveData(seconds=3, beacon="0117C55D14E4")

In [8]:
data = formatData(data)

In [9]:
data[scanners] = minMaxScaling(data[scanners])

In [10]:
data = data.rolling(15, min_periods=1).mean().reset_index()

In [11]:
data = data.ffill().fillna(0)

In [12]:
pred = model.predict(data[scanners].values)

In [13]:
pred = enc.inverse_transform(pred)

In [14]:
x = np.vectorize(lambda x: pin[x]["X"])(pred).reshape((-1, 1))
y = np.vectorize(lambda x: pin[x]["Y"])(pred).reshape((-1, 1))

In [15]:
np.expand_dims(np.hstack((x, y)).mean(axis=0), axis=0)

array([[25.94528031, 10.20101431]])